In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

#API Key
from api_keys import g_key


In [2]:
#load in CSV from part 1
weather_df = pd.read_csv("../WeatherPy/Data/city_data.csv")
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mossamedes,0,AO,1626970311,58,-15.1961,12.1522,72.54,8.16
1,Ereymentau,0,KZ,1626970312,42,51.6199,73.1033,68.99,6.24
2,Kitob,0,UZ,1626970312,14,39.0843,66.8332,86.58,1.72
3,Vila Velha,75,BR,1626970312,70,-20.3297,-40.2925,75.96,12.66
4,Östersund,75,SE,1626970313,67,63.1792,14.6357,55.85,29.93
...,...,...,...,...,...,...,...,...,...
533,Cadiz,0,ES,1626970478,21,36.5000,-5.7500,99.12,11.32
534,Nadym,52,RU,1626970479,58,65.5333,72.5167,58.37,8.75
535,Westport,20,US,1626970479,55,41.1415,-73.3579,81.86,1.01
536,Zhuji,14,CN,1626970479,98,29.7188,120.2423,80.51,3.87


In [3]:
# Humidity Heatmap
# use configure to import maps
gmaps.configure(api_key=g_key)

# use Lat and Lngs for locations, store in a list to use later
locations = weather_df[["Lat", "Lng"]]

# use humidity and store in list for use in heat map values
humidity = weather_df[["Humidity"]]



In [4]:
# humidity = weather_df[["Humidity"]]

# # Plot Heatmap
# heat_map = gmaps.figure(center=(0, 0), zoom_level=2)
# max_intensity = np.max(humidity)


# # Heat layer...
# heat = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# # Apply Layer
# heat_map.add_layer(heat)

# #generate display
# heat_map

humidity = weather_df["Humidity"]

fig = gmaps.figure(center=(0, 0), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

In [6]:
# New df for weather

In [7]:
# Using example criteria to narrow down city df
narrow_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] >= 0) &
                          (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

narrow_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mossamedes,0,AO,1626970311,58,-15.1961,12.1522,72.54,8.16
7,Martapura,90,ID,1626970314,94,-3.4167,114.8500,77.09,2.30
10,Arteaga,74,MX,1626970315,78,18.4667,-102.4167,71.53,3.33
22,Nanfeng,1,CN,1626970319,92,23.7434,111.7959,78.44,0.49
29,Caucayá,90,CO,1626970256,91,-0.1934,-74.7819,76.60,3.89
...,...,...,...,...,...,...,...,...,...
512,Taos,1,US,1626970473,57,36.4072,-105.5731,75.72,1.01
517,Palu,99,ID,1626970474,92,-0.8917,119.8707,76.50,2.55
520,Troitskoye,100,RU,1626970475,62,52.9821,84.6757,70.50,9.19
524,Along,10,IN,1626970476,91,28.1667,94.7667,77.27,2.59


In [8]:
# Hotel df

In [14]:
# df for hotel using filtered df
hotel_df = narrow_df.loc[:,["City","Country","Lat","Lng"]]

#add 'hotel' column to df
hotel_df["Hotel"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel
0,Mossamedes,AO,-15.1961,12.1522,
7,Martapura,ID,-3.4167,114.8500,
10,Arteaga,MX,18.4667,-102.4167,
22,Nanfeng,CN,23.7434,111.7959,
29,Caucayá,CO,-0.1934,-74.7819,
...,...,...,...,...,...
512,Taos,US,36.4072,-105.5731,
517,Palu,ID,-0.8917,119.8707,
520,Troitskoye,RU,52.9821,84.6757,
524,Along,IN,28.1667,94.7667,


In [15]:
# Parameters for hotel searcha and url

In [16]:
# url build
search_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# search parameters
params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

In [17]:
#for loop

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(search_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("---------------------")
        time.sleep(1)
    print("---------------------Ending Search")

Retrieving Results for Index 0: Mossamedes.
Closest hotel in Mossamedes is Hotel Chik Chik Namibe.
---------------------Ending Search
Retrieving Results for Index 7: Martapura.
Closest hotel in Martapura is Grand Dafam Q Hotel Banjarbaru || DHM Syariah.
---------------------Ending Search
Retrieving Results for Index 10: Arteaga.
Could not find result
---------------------
---------------------Ending Search
Retrieving Results for Index 22: Nanfeng.
Closest hotel in Nanfeng is Jinhong Hotel.
---------------------Ending Search
Retrieving Results for Index 29: Caucayá.
Closest hotel in Caucayá is la casona de juancho.
---------------------Ending Search
Retrieving Results for Index 30: Charleville-Mézières.
Closest hotel in Charleville-Mézières is ibis Charleville-Mézières.
---------------------Ending Search
Retrieving Results for Index 33: Guerrero Negro.
Closest hotel in Guerrero Negro is The Halfway Inn.
---------------------Ending Search
Retrieving Results for Index 34: Jamestown.
Close

Closest hotel in Portage is Rocking Horse Inn.
---------------------Ending Search
Retrieving Results for Index 327: Hashimoto.
Closest hotel in Hashimoto is Route Inn Hashimoto Hotel.
---------------------Ending Search
Retrieving Results for Index 329: Hamilton.
Closest hotel in Hamilton is Graduate Cincinnati.
---------------------Ending Search
Retrieving Results for Index 332: Asău.
Closest hotel in Asău is Pensiunea Sophia.
---------------------Ending Search
Retrieving Results for Index 334: Vila Franca do Campo.
Closest hotel in Vila Franca do Campo is Pestana Bahia Praia.
---------------------Ending Search
Retrieving Results for Index 342: Caravelas.
Closest hotel in Caravelas is Pousada Encanto Abrolhos.
---------------------Ending Search
Retrieving Results for Index 346: Ampanihy.
Closest hotel in Ampanihy is Angora Hotel.
---------------------Ending Search
Retrieving Results for Index 351: Tonj.
Could not find result
---------------------
---------------------Ending Search
Retr

In [18]:
hotel_df

,City,Country,Lat,Lng,Hotel,Hotel Name
0,Mossamedes,AO,-15.1961,12.1522,,Hotel Chik Chik Namibe
7,Martapura,ID,-3.4167,114.8500,,Grand Dafam Q Hotel Banjarbaru || DHM Syariah
10,Arteaga,MX,18.4667,-102.4167,,NaN
22,Nanfeng,CN,23.7434,111.7959,,Jinhong Hotel
29,Caucayá,CO,-0.1934,-74.7819,,la casona de juancho
...,...,...,...,...,...,...
512,Taos,US,36.4072,-105.5731,,Hotel La Fonda de Taos
517,Palu,ID,-0.8917,119.8707,,Palu Golden Hotel
520,Troitskoye,RU,52.9821,84.6757,,NaN
524,Along,IN,28.1667,94.7667,,Reyi Homestay


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
marks = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marks)

#generate map
fig


Figure(layout=FigureLayout(height='420px'))